In [5]:
import numpy as np

In [6]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [7]:

X_train, y_train = load_mnist('data/fashion', kind='train')
X_test, y_test = load_mnist('data/fashion', kind='t10k')

In [8]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [ ]:
# Classes of Fashion MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [9]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [11]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

c:\Users\Daniel\anaconda3\envs\py310\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [12]:
y_pred_train = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred_train)

0.8748833333333333

In [13]:
y_pred_test = lin_clf.predict(X_test_scaled)
accuracy_score(y_test, y_pred_test)

0.837

In [14]:
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

In [15]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.8727333333333334

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

In [17]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ....C=7.802523805502085, gamma=0.026004208220401302; total time=   0.1s
[CV] END ....C=7.802523805502085, gamma=0.026004208220401302; total time=   0.1s
[CV] END ....C=7.802523805502085, gamma=0.026004208220401302; total time=   0.1s
[CV] END ...C=10.462031978044882, gamma=0.004746820427251581; total time=   0.1s
[CV] END ...C=10.462031978044882, gamma=0.004746820427251581; total time=   0.1s
[CV] END ...C=10.462031978044882, gamma=0.004746820427251581; total time=   0.1s
[CV] END ...C=3.5806304211739937, gamma=0.022885666552726436; total time=   0.1s
[CV] END ...C=3.5806304211739937, gamma=0.022885666552726436; total time=   0.1s
[CV] END ...C=3.5806304211739937, gamma=0.022885666552726436; total time=   0.1s
[CV] END ....C=3.384695624138363, gamma=0.015138722017441557; total time=   0.1s
[CV] END ....C=3.384695624138363, gamma=0.015138722017441557; total time=   0.1s
[CV] END ....C=3.384695624138363, gamma=0.015138

RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000217830B2D70>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000217830B1EA0>},
                   verbose=2)

In [18]:
rnd_search_cv.best_estimator_

SVC(C=10.462031978044882, gamma=0.004746820427251581)

In [19]:
rnd_search_cv.best_score_

0.7770255285225346

In [20]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=10.462031978044882, gamma=0.004746820427251581)

In [21]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9999333333333333

In [22]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.883